# Lyric baseline

In [1]:
from src.baseline import make_all_lyric_baselines

# Generate all baselines with summary
stats = make_all_lyric_baselines()

Generating lyric baselines for 40 victory odes...


Processing odes: 100%|██████████| 40/40 [02:06<00:00,  3.16s/it]


LYRIC BASELINE GENERATION SUMMARY
Total odes processed: 40/40

Total lines generated: 337,200

Source breakdown:
  Pindar corpus: 334,600 (99.2%)
  External corpus: 2,600 (0.8%)

Modification breakdown:
  Unaltered: 322,592 (95.7%)
  Trimmed: 14,608 (4.3%)
  Padded: 0 (0.0%)


In [1]:
# Test the fixed external corpus responsion_id handling
from src.baseline import load_cached_lyric_corpus, lyric_line_sample_cached

# Load the cached corpus
cached_corpus = load_cached_lyric_corpus("data/cached_lyric_corpus.pkl", "data/compiled/triads")

target_length = 30
print(f"=== TESTING FIXED EXTERNAL CORPUS FOR LENGTH {target_length} ===")

# Test the fixed system
used_responsions_this_position = set()
unique_samples_found = 0
used_metrical_positions = set()

print("Testing 10 samples to see variety of responsion_ids from external corpus:")
for attempt in range(10):
    result = lyric_line_sample_cached(
        length=target_length,
        cached_corpus=cached_corpus, 
        seed=1453 + attempt,
        debug=False,
        exclude_file="ht_olympians_triads.xml",
        used_metrical_positions=used_metrical_positions,
        used_responsions_this_position=used_responsions_this_position
    )
    
    if result is not None:
        source_attr = result.get('source', 'No source')
        
        # Extract responsion_id for independence tracking
        if source_attr.startswith('external:'):
            # New format: "external:responsion_id, strophe X, line Y"
            responsion_id = source_attr.split(',')[0].replace('external:', '').strip()
        elif ',' in source_attr:
            # Internal format: "responsion_id, strophe X, line Y" 
            responsion_id = source_attr.split(',')[0].strip()
        else:
            # Fallback
            responsion_id = source_attr
            
        if responsion_id not in used_responsions_this_position:
            used_responsions_this_position.add(responsion_id)
            unique_samples_found += 1
            print(f"✅ Sample {unique_samples_found}: {responsion_id} (from {source_attr})")
        else:
            print(f"⚠️  Sample would repeat responsion: {responsion_id}")
    else:
        print(f"❌ Failed to find any line for attempt {attempt+1}")

print(f"\n📊 IMPROVED RESULTS:")
print(f"   Unique samples possible: {unique_samples_found}")
print(f"   Used responsions: {sorted(used_responsions_this_position)}")

if unique_samples_found >= 5:
    print(f"   🎉 SUCCESS: Can now generate {unique_samples_found} unique samples (≥5 needed)!")
else:
    print(f"   ⚠️  Still only {unique_samples_found} unique samples (need ≥5)")
    
    # Check what responsion_ids are available in external corpus
    print(f"\n=== CHECKING EXTERNAL CORPUS AVAILABILITY ===")
    external_responsions_available = set()
    
    # Manually check one external corpus file to see available responsions  
    import os
    from lxml import etree
    from src.stats import canonical_sylls
    
    external_folder = "/Users/albin/git/aristophanis-cantica/data/compiled/"
    if os.path.exists(external_folder):
        xml_files = [f for f in os.listdir(external_folder) if f.endswith('.xml')][:3]  # Check first 3 files
        print(f"Checking external files: {xml_files}")
        
        for xml_file in xml_files:
            try:
                file_path = os.path.join(external_folder, xml_file)
                tree = etree.parse(file_path)
                root = tree.getroot()
                
                for strophe in root.findall(".//strophe"):
                    responsion_id = strophe.get('responsion', 'unknown')
                    external_responsions_available.add(responsion_id)
                    
            except Exception as e:
                print(f"Could not parse {xml_file}: {e}")
        
        print(f"Available external responsion_ids: {sorted(external_responsions_available)}")
        print(f"Total external responsions available: {len(external_responsions_available)}")
    else:
        print(f"External corpus folder not found: {external_folder}")

=== TESTING FIXED EXTERNAL CORPUS FOR LENGTH 30 ===
Testing 10 samples to see variety of responsion_ids from external corpus:
✅ Sample 1: py01 (from py01, strophe 3, line 6)
⚠️  Sample would repeat responsion: eq01
⚠️  Sample would repeat responsion: eq06
⚠️  Sample would repeat responsion: eq03
⚠️  Sample would repeat responsion: eq04
⚠️  Sample would repeat responsion: av09
⚠️  Sample would repeat responsion: pax03
⚠️  Sample would repeat responsion: ra07
⚠️  Sample would repeat responsion: pax03
⚠️  Sample would repeat responsion: ra07
⚠️  Sample would repeat responsion: ach01
✅ Sample 2: is07 (from is07, strophe 2, line 5)

📊 IMPROVED RESULTS:
   Unique samples possible: 2
   Used responsions: ['ach01', 'av09', 'eq01', 'eq03', 'eq04', 'eq06', 'is07', 'pax03', 'py01', 'ra07']
   ⚠️  Still only 2 unique samples (need ≥5)

=== CHECKING EXTERNAL CORPUS AVAILABILITY ===
Checking external files: ['responsion_pax_compiled.xml', 'responsion_pl_compiled.xml', 'responsion_v_compiled.xml']
Av